In [1]:
import pandas as pd
import numpy as np

from IPython.display import display
import datetime
import plotly.express as px
from plotly.colors import sequential
from plotly.colors import diverging

from plotly.subplots import make_subplots

import math

In [2]:
matches_table = pd.read_csv('data/matches.csv')
display(matches_table)

,Date,Format,Match Number,Stage,Player 1,Player 2,Score 1,Score 2
0,2023-11-18,[Gen 9] OU,1,Group,Evan Sooklal,Will Simpson,2,6
1,2023-11-18,[Gen 9] OU,2,Group,Paul Bartenfeld,Roman Ramirez,2,6
2,2023-11-18,[Gen 9] OU,3,Group,Roman Ramirez,Evan Sooklal,6,3
3,2023-11-18,[Gen 9] OU,4,Group,Will Simpson,Paul Bartenfeld,6,1
4,2023-11-18,[Gen 9] OU,5,Group,Paul Bartenfeld,Evan Sooklal,1,6
...,...,...,...,...,...,...,...,...
121,2024-06-09,[Gen 9] RU,4b,Group,Roman Ramirez,Evan Sooklal,2,6
122,2024-06-09,[Gen 9] RU,5a,Group,Evan Sooklal,Lilith Karyadi,2,6
123,2024-06-09,[Gen 9] RU,5a,Group,Evan Sooklal,Lilith Karyadi,5,6
124,2024-06-09,[Gen 9] RU,1,Final,Evan Sooklal,Lilith Karyadi,5,6


In [3]:
updated_time = f'<i>Updated {str(datetime.datetime.now().strftime("%A, %b %d, %Y %H:%M:%S"))} CT</i>'
link_fig = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/index.html'>ELO Graph</a>"
link_fig1 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig1.html'>H2H Stats</a>"
link_fig2 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig2.html'>H2H Advanced Stats</a>"
link_fig3 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig3.html'>H2H More Stats</a>"
link_fig4 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig4.html'>Player Data</a>"
link_fig5 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig5.html'>Matches Data</a>"
link_fig6 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig6.html'>Money-Lines</a>"

In [4]:
set_player1 = set(matches_table['Player 1'])
set_player2 = set(matches_table['Player 2'])
set_players = set_player1.union(set_player2)
num_players = len(set_players)
display(set_players)

{'Aaron Carter',
 'Evan Sooklal',
 'Jack Massingill',
 'Lilith Karyadi',
 'Paul Bartenfeld',
 'Roman Ramirez',
 'Will Simpson'}

In [5]:
# elo stuff
K = 31
base = 3

def expected_score(ratingA, ratingB, base=None):
    # game variables
    # base = 10
    normal_elo_difference = 100

    return (1 / (1 + np.power(base, (ratingB - ratingA) / normal_elo_difference)))

def rating_change(score, expected_score, K=None):
    # k-factor: determines how strongly a result affects the rating change
    # usually between 10 and 40, bit with a lot of games, we want to change it often
    # K = 32
    return K * (score - expected_score)

def sort_index_by_elo(table, elo):
    for i in range(2):
        table.sort_index(axis=i, key=(lambda x: [elo[y] for y in x.values]), inplace=True)

def add_elo_to_index(table):
    table.index= [f"{x}<br>{(elo[x]):,.0f}" for x in table.index]
    table.columns = [f"{x}<br>({(elo[x]):,.0f})" for x in table.columns]

def generate_elo(K, base):
    # ELO INITIALIZATION
    starting_elo = 1200.0
    elo = dict.fromkeys(set_players, starting_elo)
    elo_time = np.zeros([len(matches_table) + 1, num_players])
    elo_time_table = pd.DataFrame(elo_time)
    elo_time_table.columns = sorted(elo)
    elo_time_table.replace(0, np.NaN, inplace=True)

    prev_elo_time_table = elo_time_table.copy(deep=True)
    d_elo_time_table = elo_time_table.copy(deep=True)
    exp_elo_time_table = elo_time_table.copy(deep=True)
    wl_time_table = elo_time_table.copy(deep=True)

    record_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]),dtype=object)
    record_table.index = list(set_players)
    record_table.columns = list(set_players)

    luck_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]))
    luck_table.index = list(set_players)
    luck_table.columns = list(set_players)

    for c in record_table.columns:
        for r in record_table.index:
            record_table.at[r, c] = np.array([0, 0])

    plus_minus_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]),dtype=object)
    plus_minus_table.index = list(set_players)
    plus_minus_table.columns = list(set_players)
    
    for c in plus_minus_table.columns:
        for r in plus_minus_table.index:
            plus_minus_table.at[r, c] = np.array([0, 0])

    elo_time_table.iloc[0,:] = starting_elo
    prev_elo_time_table.iloc[0,:] = starting_elo

    for (i, row) in matches_table.iterrows():
        elo_p1 = elo[row['Player 1']]
        elo_p2 = elo[row['Player 2']]

        prev_elo_time_table.loc[i + 1, row['Player 1']] = elo_p1
        prev_elo_time_table.loc[i + 1, row['Player 2']] = elo_p2
        
        win_prob_p1 = expected_score(elo_p1, elo_p2, base=base)
        win_prob_p2 = expected_score(elo_p2, elo_p1, base=base)
        
        exp_elo_time_table.loc[i + 1, row['Player 1']] = win_prob_p1
        exp_elo_time_table.loc[i + 1, row['Player 2']] = win_prob_p2

        rating_change_p1 = rating_change(row['Score 1'] > row['Score 2'], win_prob_p1, K=K)
        rating_change_p2 = rating_change(row['Score 2'] > row['Score 1'], win_prob_p2, K=K)

        if rating_change_p1 > 0:
            luck_table.loc[row['Player 1'], row['Player 2']] += 1 - win_prob_p1
            luck_table.loc[row['Player 2'], row['Player 1']] += 0 - win_prob_p2
        elif rating_change_p2 > 0:
            luck_table.loc[row['Player 1'], row['Player 2']] += 0 - win_prob_p1
            luck_table.loc[row['Player 2'], row['Player 1']] += 1 - win_prob_p2

        plus_minus_table.loc[row['Player 1'], row['Player 2']][0] += row['Score 1']
        plus_minus_table.loc[row['Player 1'], row['Player 2']][1] += row['Score 2']
        plus_minus_table.loc[row['Player 2'], row['Player 1']][0] += row['Score 2']
        plus_minus_table.loc[row['Player 2'], row['Player 1']][1] += row['Score 1']

        if row['Score 1'] > row['Score 2']:
            record_table.loc[row['Player 1'], row['Player 2']][0] += 1
            record_table.loc[row['Player 2'], row['Player 1']][1] += 1
            wl_time_table.loc[i + 1, row['Player 1']] = row['Score 1'] - row['Score 2']
            wl_time_table.loc[i + 1, row['Player 2']] = row['Score 2'] - row['Score 1']
        elif row['Score 2'] > row['Score 1']:
            record_table.loc[row['Player 2'], row['Player 1']][0] += 1
            record_table.loc[row['Player 1'], row['Player 2']][1] += 1
            wl_time_table.loc[i + 1, row['Player 1']] = row['Score 1'] - row['Score 2']
            wl_time_table.loc[i + 1, row['Player 2']] = row['Score 2'] - row['Score 1']

        d_elo_time_table.loc[i + 1, row['Player 1']] = rating_change_p1
        d_elo_time_table.loc[i + 1, row['Player 2']] = rating_change_p2

        elo[row['Player 1']] += rating_change_p1
        elo[row['Player 2']] += rating_change_p2

        elo_time_table.loc[i + 1, row['Player 1']] = elo[row['Player 1']]
        elo_time_table.loc[i + 1, row['Player 2']] = elo[row['Player 2']]

    prev_elo_time_table = prev_elo_time_table.iloc[1:,:]
    elo_time_table = elo_time_table.iloc[1:,:]
    d_elo_time_table = d_elo_time_table.iloc[1:,:]
    exp_elo_time_table = exp_elo_time_table.iloc[1:,:]
    wl_time_table = wl_time_table.iloc[1:,:]

    list_players = sorted(set_players, key=lambda x: elo[x], reverse=False)

    for table in [record_table, luck_table, plus_minus_table]:
        sort_index_by_elo(table, elo)

    return (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, wl_time_table, record_table, luck_table, plus_minus_table)

(elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, wl_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)

luck_per_game_table = luck_table.copy(deep=True)

for c in luck_per_game_table.columns:
    for r in luck_per_game_table.index:
        if r !=c :
            luck_per_game_table.loc[r, c] /= record_table.loc[r, c].sum()
        else:
            luck_per_game_table.loc[r, c] = np.NaN


display(record_table)
display(luck_per_game_table)
display(plus_minus_table)
display(wl_time_table)

prev_elo_time_table.to_excel('exports/prev_elo_time_table.xlsx')
d_elo_time_table.to_excel('exports/d_elo_time_table.xlsx')
elo_time_table.to_excel('exports/elo_time_table.xlsx')
exp_elo_time_table.to_excel('exports/exp_elo_time_table.xlsx')



C:\Users\justr\AppData\Local\Temp\ipykernel_32696\2719908281.py:131: RuntimeWarning: invalid value encountered in scalar divide
  luck_per_game_table.loc[r, c] /= record_table.loc[r, c].sum()


,Paul Bartenfeld,Jack Massingill,Evan Sooklal,Roman Ramirez,Lilith Karyadi,Will Simpson,Aaron Carter
Paul Bartenfeld,"[0, 0]","[0, 0]","[1, 5]","[2, 4]","[0, 0]","[0, 6]","[1, 3]"
Jack Massingill,"[0, 0]","[0, 0]","[0, 2]","[0, 2]","[0, 0]","[1, 1]","[0, 2]"
Evan Sooklal,"[5, 1]","[2, 0]","[0, 0]","[10, 5]","[0, 3]","[5, 12]","[3, 10]"
Roman Ramirez,"[4, 2]","[2, 0]","[5, 10]","[0, 0]","[0, 2]","[8, 7]","[5, 9]"
Lilith Karyadi,"[0, 0]","[0, 0]","[3, 0]","[2, 0]","[0, 0]","[0, 3]","[0, 0]"
Will Simpson,"[6, 0]","[1, 1]","[12, 5]","[7, 8]","[3, 0]","[0, 0]","[4, 10]"
Aaron Carter,"[3, 1]","[2, 0]","[10, 3]","[9, 5]","[0, 0]","[10, 4]","[0, 0]"


,Paul Bartenfeld,Jack Massingill,Evan Sooklal,Roman Ramirez,Lilith Karyadi,Will Simpson,Aaron Carter
Paul Bartenfeld,NaN,NaN,-0.179256,-0.054102,NaN,-0.301516,-0.061861
Jack Massingill,NaN,NaN,-0.504238,-0.438942,NaN,0.238515,-0.115022
Evan Sooklal,0.179256,0.504238,NaN,0.151665,-0.468953,-0.119322,-0.125188
Roman Ramirez,0.054102,0.438942,-0.151665,NaN,-0.645015,0.130468,0.030219
Lilith Karyadi,NaN,NaN,0.468953,0.645015,NaN,-0.622451,NaN
Will Simpson,0.301516,-0.238515,0.119322,-0.130468,0.622451,NaN,-0.173710
Aaron Carter,0.061861,0.115022,0.125188,-0.030219,NaN,0.173710,NaN


,Paul Bartenfeld,Jack Massingill,Evan Sooklal,Roman Ramirez,Lilith Karyadi,Will Simpson,Aaron Carter
Paul Bartenfeld,"[0, 0]","[0, 0]","[24, 35]","[21, 31]","[0, 0]","[16, 36]","[15, 21]"
Jack Massingill,"[0, 0]","[0, 0]","[3, 12]","[1, 12]","[0, 0]","[6, 6]","[7, 12]"
Evan Sooklal,"[35, 24]","[12, 3]","[0, 0]","[64, 55]","[12, 18]","[53, 83]","[45, 66]"
Roman Ramirez,"[31, 21]","[12, 1]","[55, 64]","[0, 0]","[6, 12]","[67, 54]","[52, 61]"
Lilith Karyadi,"[0, 0]","[0, 0]","[18, 12]","[12, 6]","[0, 0]","[12, 18]","[0, 0]"
Will Simpson,"[36, 16]","[6, 6]","[83, 53]","[54, 67]","[18, 12]","[0, 0]","[35, 59]"
Aaron Carter,"[21, 15]","[12, 7]","[66, 45]","[61, 52]","[0, 0]","[59, 35]","[0, 0]"


,Aaron Carter,Evan Sooklal,Jack Massingill,Lilith Karyadi,Paul Bartenfeld,Roman Ramirez,Will Simpson
1,NaN,-4.0,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,-4.0,4.0,NaN
3,NaN,-3.0,NaN,NaN,NaN,3.0,NaN
4,NaN,NaN,NaN,NaN,-5.0,NaN,5.0
5,NaN,5.0,NaN,NaN,-5.0,NaN,NaN
...,...,...,...,...,...,...,...
122,NaN,4.0,NaN,NaN,NaN,-4.0,NaN
123,NaN,-4.0,NaN,4.0,NaN,NaN,NaN
124,NaN,-1.0,NaN,1.0,NaN,NaN,NaN
125,NaN,-1.0,NaN,1.0,NaN,NaN,NaN


In [6]:
# player stats

player_table = pd.DataFrame(np.zeros([len(set_players), 0]))
player_table.index = set_players
player_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), ascending=False, inplace=True)

# ELO

player_table['Current ELO'] = [elo[x] for x in player_table.index]
player_table['Max ELO'] = elo_time_table.replace(to_replace=np.NaN, value=-np.inf).max(axis=0)
player_table['Min ELO'] = elo_time_table.replace(to_replace=np.NaN, value=np.inf).min(axis=0)

date2matches_len = dict.fromkeys(set(matches_table['Date']),0)
date2matches_first = dict.fromkeys(set(matches_table['Date']),0)
for (i, row) in matches_table.iterrows():
    if date2matches_len[row['Date']] == 0:
        date2matches_first[row['Date']] = i
    date2matches_len[row['Date']] += 1

# CHAMPIONSHIPS AND RUNNER UPS

championships = dict.fromkeys(set_players, 0)
runner_ups = dict.fromkeys(set_players, 0)
for date in set(matches_table['Date']):
    row = matches_table.iloc[date2matches_first[date] + date2matches_len[date] - 1, :]
    if row['Score 1'] > row['Score 2']:
        championships[row['Player 1']] += 1
        runner_ups[row['Player 2']] += 1
    elif row['Score 2'] > row['Score 1']:
        championships[row['Player 2']] += 1
        runner_ups[row['Player 1']] += 1

player_table['Titles'] = pd.DataFrame(championships.values(), index=championships.keys())
player_table['Runner-Ups'] = pd.DataFrame(runner_ups.values(), index=runner_ups.keys())

# RECORDS

player_table['Record'] = record_table.sum(axis=1)
player_table['GP'] = [sum(r) for r in player_table['Record']]
player_table['W'] = [r[0] for r in player_table['Record']]
player_table['L'] = [r[1] for r in player_table['Record']]
player_table['WL'] = player_table['W'] - player_table['L']
player_table['GB'] = (player_table['WL'] - player_table['WL'].max()) / 2
player_table['PCT'] = player_table['W'] / player_table['GP']
player_table.drop(columns='Record', inplace=True)

# POINTS

player_table['Raw +/-'] = plus_minus_table.sum(axis=1)
player_table['PD'] = [r[0] - r[1] for r in player_table['Raw +/-']]
player_table['PF'] = [r[0] for r in player_table['Raw +/-']]
player_table['PA'] = [r[1] for r in player_table['Raw +/-']]
player_table['PD/G'] = player_table['PD'] / player_table['GP']
player_table['PF/G'] = player_table['PF'] / player_table['GP']
player_table['PA/G'] = player_table['PA'] / player_table['GP']
player_table.drop(columns='Raw +/-', inplace=True)

# LUCK

player_table['Luck'] = luck_table.sum(axis=1)
player_table['Luck/G'] = player_table['Luck'] / player_table['GP']

# N-POINT GAMES
npg_table = pd.DataFrame(np.zeros([len(set_players), 0]))
npg_table.index = player_table.index
sort_index_by_elo(npg_table, elo)

for i in range(6):
    npg_table[f'{i+1}-PT G'] = [[0, 0] for x in range(len(set_players))]
for (i, row) in matches_table.iterrows():
    npg_table.loc[row['Player 1'], f"{abs(row['Score 1'] - row['Score 2'])}-PT G"][row['Score 1'] < row['Score 2']] += 1
    npg_table.loc[row['Player 2'], f"{abs(row['Score 1'] - row['Score 2'])}-PT G"][row['Score 1'] > row['Score 2']] += 1

# STREAKS

streaks = dict.fromkeys(set_players, list())

for player in streaks.keys():
    running = [0]
    for is_win in wl_time_table[player].dropna().reset_index(drop=True).apply(np.sign):
        if running[-1] == 0:
            running.append(is_win)
        elif (is_win > 0) and (running[-1] < 0):
            running.append(1)
        elif (is_win < 0) and (running[-1] > 0):
            running.append(-1)
        else:
            running.append(running[-1] + is_win)
    streaks[player] = running[1:]

streaks_table = pd.DataFrame(np.zeros([max([len(x) for x in streaks.values()]), len(set_players)]))
streaks_table.columns = set_players
streaks_table.sort_index(axis=1, key=(lambda x: [elo[y] for y in x.values]), ascending=False, inplace=True)

for player in streaks_table.columns:
    streaks_table[player] = pd.Series(streaks[player])

for x in ['1-PT G', '6-PT G']:
    player_table[x] = npg_table[x]
    player_table[x] = [f'{w}-{l}' for (w, l) in player_table[x]]


player_table['Streak'] = 0
for player in player_table.index:
    val = ""
    if streaks[player][-1] > 0:
        val += "W"
    elif streaks[player][-1] < 0:
        val += "L"
    val += str(int(np.abs(streaks[player][-1])))
    player_table.loc[player, 'Streak'] = val

player_table['L10'] = ""
for player in player_table.index:
    l10_w = 0
    l10_l = 0
    for i in range(10):
        try:
            if streaks[player][-1 - i] > 0:
                l10_w += 1
            elif streaks[player][-1 - i] < 0:
                l10_l += 1
        except IndexError:
            pass
    player_table.loc[player, 'L10'] = f"{l10_w}-{l10_l}"

player_table.drop('WL', axis=1, inplace=True)
player_table.drop('Luck', axis=1, inplace=True)

# STYLIZING

from pandas.io.formats.style import Styler

player_table_styled = Styler(player_table, cell_ids=False).format(
    {'Current ELO': '{:.0f}',
    'Max ELO': '{:.0f}',
    'Min ELO': '{:.0f}',
    'WL': '{:+,.0f}',
    'GB': '{:,.1f}',
    'PCT': '{:,.3f}',
    'PD': '{:+.0f}',
    'PD/G': '{:+,.1f}',
    'PF/G': '{:,.1f}',
    'PA/G': '{:,.1f}',
    'Luck': '{:+,.1%}',
    'Luck/G': '{:+,.1%}'
    }
)

player_table_styled.set_caption("Player Table Statistics")

styles = [
    {
        'selector': '',
        'props': [
            ('text-align', 'center'),
            ('font-size', '11pt'),
            ('border-collapse', 'collapse'),#'separated'),
            ('border', '1px solid silver')
        ]
    },
    {
        'selector': 'caption',
        'props': [('font', 'bold 20px Verdana')]
    },
    {
        'selector': 'tr:nth-child(even)',
        'props': 'background-color: #D3D3D3'
    },
    {
        'selector': 'td:hover',
        'props': [('background-color','#ffffb3')]
    },
    {
        'selector': 'th',
        'props': [
            ('font-family', 'Verdana'),
            ('background-color', '#828282'),
            ('color', '#FFFFFF'),
            ('padding-left', '5px'),
            ('padding-right', '5px')
        ]
    },
    {
        'selector': 'td',
        'props': [
            ('font-family', 'Courier New'),
            ('font-weight', 'bold'),
            ('padding-left', '5px'),
            ('padding-right', '5px')
        ]
    }
]

player_table_styled.set_table_styles(styles, overwrite=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(player_table_styled)

player_table_styled.to_html('figures/fig4.html')

matches_table_styled = Styler(matches_table, cell_ids=False)
matches_table_styled.set_caption("Matches Table")
matches_table_styled.set_table_styles(styles, overwrite=True)
matches_table_styled.to_html('figures/fig5.html')



,Current ELO,Max ELO,Min ELO,Titles,Runner-Ups,GP,W,L,GB,PCT,PD,PF,PA,PD/G,PF/G,PA/G,Luck/G,1-PT G,6-PT G,Streak,L10
Aaron Carter,1328,1341,1182,3,1,47,34,13,0.0,0.723,+65,219,154,+1.4,4.7,3.3,+8.8%,7-5,1-1,W2,7-3
Will Simpson,1226,1307,1168,3,2,57,33,24,-6.0,0.579,+19,232,213,+0.3,4.1,3.7,+1.5%,6-3,1-4,W3,4-6
Lilith Karyadi,1226,1244,1201,0,1,8,5,3,-9.5,0.625,+6,42,36,+0.8,5.2,4.5,+10.4%,2-2,0-0,L1,5-3
Roman Ramirez,1201,1307,1118,1,1,54,24,30,-13.5,0.444,+10,223,213,+0.2,4.1,3.9,+0.0%,2-7,4-0,L6,4-6
Evan Sooklal,1178,1252,1149,0,2,56,25,31,-13.5,0.446,-28,221,249,-0.5,3.9,4.4,-1.3%,9-8,1-0,L3,4-6
Jack Massingill,1149,1181,1127,0,0,8,1,7,-13.5,0.125,-25,17,42,-3.1,2.1,5.2,-20.5%,0-0,1-3,W1,1-7
Paul Bartenfeld,1093,1184,1093,0,0,22,4,18,-17.5,0.182,-47,76,123,-2.1,3.5,5.6,-15.7%,1-2,0-0,L8,2-8


In [7]:
import plotly.graph_objects as go
import plotly.colors as pc

fig = go.Figure()

# adding player
ranking = 1
trace_colors = pc.qualitative.Bold
for (i, (player, current_rating)) in enumerate(sorted(elo.items(), key=lambda x:x[1], reverse=True)):
    fig.add_trace(go.Scatter(
        x=elo_time_table.index,
        y=elo_time_table[player],
        name=f'#{ranking} ({current_rating:.0f}) {player}',
        mode='lines+markers',
        connectgaps=True,
        text=[
            f"<br><b>Pre-Game ELO: </b>{x[1]:.0f}<br><b>Win Probability: </b>{x[0]:.1%}<br><br><b>Change in ELO:</b> {x[2]:+.0f}<br>" for x in zip(
                exp_elo_time_table[player],
                prev_elo_time_table[player],
                d_elo_time_table[player]
            )
        ],
        line=dict(
            shape='hv',
            color=trace_colors[i % len(trace_colors)]
        )
    ))
    ranking += 1

# adding highlighting by tournament
tournaments = list(sorted(set(matches_table['Date'])))
vrect_colors = ['green', 'red', 'yellow', 'blue', 'orange']
for (i, tourney) in enumerate(tournaments):
    fig.add_vrect(
        annotation_text=tourney,
        annotation_position="top left",
        x0=matches_table['Date'][matches_table['Date'] == tourney].index[0] + 0.5,
        x1=matches_table['Date'][matches_table['Date'] == tourney].index[-1] + 1.5,
        fillcolor=vrect_colors[i % len(vrect_colors)],
        opacity=0.1,
        line_width=0
    )

fig.update_layout(
    title=f'<b>Pokémon Showdown ELO Rating System by Roman Ramirez</b><br>{updated_time}<br>{link_fig1}, {link_fig2}, {link_fig3}, {link_fig4}, {link_fig5}, {link_fig6}',
    xaxis_title='<b>Game Number</b>',
    yaxis_title='<b>ELO Rating</b>'
)

customdata = np.stack((
        list(matches_table['Player 1']),
        list(matches_table['Player 2']),
        list(matches_table['Score 1']),
        list(matches_table['Score 2']),
        matches_table['Date'],
        list(matches_table['Format']),
        list(matches_table['Match Number'])
    ), axis=-1)
hovertemplate = (
    '<i>%{customdata[4]|%A, %B %d, %Y}, Game %{x}</i><br>' +
    '<b>%{fullData.name}</b><br><br>' + 
    '<b>Format:</b> %{customdata[5]}<br>' + 
    '<b>Tournament Match:</b> %{customdata[6]}<br><br>' + 
    '<b>%{customdata[0]} vs. %{customdata[1]}</b><br>' +
    '<b>Final Score:</b> %{customdata[2]}-%{customdata[3]}<br>' + 
    '%{text}' + 
    '<b>Post-Game ELO:</b> %{y:,.0f}<br>' +
    '<extra></extra>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate,
    opacity=0.8,
    legendgrouptitle_text='<b>#<i>Rank</i> (<i>Current ELO</i>) <i>Player</i></b>'
)

fig.show()
fig.write_html("index.html")

In [8]:
# historical table, with record_table
win_pct_table = record_table.copy(deep=True)
text_record_table = record_table.copy(deep=True)

for c in win_pct_table.columns:
    for r in win_pct_table.index:
        if c == r:
            win_pct_table.loc[r, c] = np.NaN
            text_record_table.loc[r, c] = ""
        else:
            w, l = win_pct_table.loc[r, c]
            w_pct = (w) / (w + l) if (w + l) != 0 else 0
            win_pct_table.loc[r, c] = w_pct
            text_record_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"


for c in record_table.columns:
    for r in record_table.index:
        if sum(record_table.loc[r, c]) == 0:
            text_record_table.loc[r, c] = ""
            win_pct_table.loc[r, c] = np.NaN

win_prob_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]), dtype=object)
win_prob_table.columns = set_players
win_prob_table.index = set_players
sort_index_by_elo(win_prob_table, elo)

for c in win_prob_table.columns:
    for r in win_prob_table.index:
            if r != c:
                win_prob_table.loc[r, c] = expected_score(elo[r], elo[c], base=base)
            else:
                win_prob_table.loc[r, c] = np.NaN


In [9]:
fig1 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f"<b>{x}</b>" for x in ["Historical Win Percentage", "Current Matchup Predictor"]],
    x_title=f"{link_fig}, {link_fig2}, {link_fig3}, {link_fig4}, {link_fig5}, {link_fig6}"
)

go_win_pct = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=win_pct_table,
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_record_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

go_win_prob = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=win_prob_table,
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=win_prob_table.applymap(lambda x: "" if np.isnan(x) else f"{x:.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig1.add_trace(go_win_pct, row=1, col=1)
fig1.add_trace(go_win_prob, row=1, col=2)

fig1.layout.yaxis1.title="<b>Self</b>"
fig1.layout.yaxis2.title="<b>Self</b>"
fig1.layout.xaxis1.title="<b>Other</b>"
fig1.layout.xaxis2.title="<b>Other</b>"

fig1.update_traces(hoverinfo='skip')

fig1.update_layout(
    title=f"<b>Head-to-Head Statistics</b>, {updated_time}"
)

fig1.show()
fig1.write_html('figures/fig1.html')

In [10]:
fig3 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f"<b>{x}</b>" for x in ["Head-to-Head Plus Minus", "N-Game Historical Record"]],
    x_title=f"<br>{link_fig}, {link_fig1}, {link_fig2}, {link_fig4}, {link_fig5}, {link_fig6}"
)

text_npg_table = npg_table.copy(deep=True)

for c in npg_table.columns:
    for r in npg_table.index:
        if c == r:
            text_npg_table.loc[r, c] = ""
        else:
            w, l = npg_table.loc[r, c]
            if (w + l) == 0:
                text_npg_table.loc[r, c] = ""
            else:
                w_pct = (w) / (w + l) if (w + l) != 0 else 0
                text_npg_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"

def calc_win_pct(wl):
    if sum(wl) == 0:
        return np.NaN
    else:
        return (wl[0]) / (sum(wl))

go_ng = go.Heatmap(
    x=[x.replace(" ", "<br>")+"ames" for x in npg_table.columns],
    y=[x.replace(" ", "<br>") for x in npg_table.index],
    z=npg_table.applymap(calc_win_pct),
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_npg_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

text_plus_minus_table = plus_minus_table.copy(deep=True)

for c in plus_minus_table.columns:
    for r in plus_minus_table.index:
        if c == r:
            text_plus_minus_table.loc[r, c] = ""
        else:
            w, l = plus_minus_table.loc[r, c]
            if (w + l) == 0:
                text_plus_minus_table.loc[r, c] = ""
            else:
                w_pct = (w) / (w + l) if (w + l) != 0 else 0
                text_plus_minus_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"

go_pm = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in plus_minus_table.columns],
    y=[x.replace(" ", "<br>") for x in plus_minus_table.index],
    z=plus_minus_table.applymap(calc_win_pct),
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_plus_minus_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig3.add_trace(go_pm, row=1, col=1)
fig3.add_trace(go_ng, row=1, col=2)

fig3.layout.yaxis1.title="<b>Self</b>"
fig3.layout.yaxis2.title="<b>Self</b>"
fig3.layout.xaxis1.title="<b>Other</b>"
fig3.layout.xaxis2.title="<b>Other</b>"

fig3.update_traces(
    hoverinfo='skip'
)

fig3.update_layout(
    title=f"<b>Head-to-Head Statistics</b>, {updated_time}"
)

fig3.show()
fig3.write_html('figures/fig3.html')

In [11]:
# difference in win percentage and matchup predictor
# positive number means overrated, negative number means underrated
# understanding: Player A is overrated/underrated against Player B

key_upset_table = -win_pct_table + win_prob_table
        
for c in record_table.columns:
    for r in record_table.index:
        if sum(record_table.loc[r, c]) == 0:
            key_upset_table.loc[r, c] = np.NaN


fig2 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        "<b>Over/Underrated</b><br>How Much Better is the Win Probability Against Record", 
        "<b>Average Luck Per Game</b><br>Average Difference in Actual and Expected Performance"
    ],
    x_title=f"{link_fig}, {link_fig1}, {link_fig3}, {link_fig4}, {link_fig5}, {link_fig6}"
)

go_ou = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=key_upset_table,
    zmax=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None), abs(min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)))),
    zmid=0,
    zmin=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None), abs(min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)))) * -1,
    text=key_upset_table.applymap(lambda x: "" if np.isnan(x) else f"{x:+.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

go_luck_pg = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=luck_per_game_table,
    zmax=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None), abs(min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)))),
    zmid=0,
    zmin=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None), abs(min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)))) * -1,
    text=luck_per_game_table.applymap(lambda x: "" if np.isnan(x) else f"{x:+.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig2.add_trace(go_ou, row=1, col=1)
fig2.add_trace(go_luck_pg, row=1, col=2)

fig2.layout.yaxis1.title="<b>Self</b>"
fig2.layout.yaxis2.title="<b>Self</b>"
fig2.layout.xaxis1.title="<b>Other</b>"
fig2.layout.xaxis2.title="<b>Other</b>"

fig2.update_traces(hoverinfo='skip')

fig2.update_layout(
    title=f"<b>Head-to-Head Advanced Statistics</b>, {updated_time}"
)

fig2.show()
fig2.write_html('figures/fig2.html')

In [12]:
# betting/gambling table

# to-do
# odds to win championship
# record for/against the spread
# player vs. player spread
# over/under total

odds_table = win_prob_table / win_prob_table.T
ml_table = odds_table.copy(deep=True)

for c in ml_table.columns:
    for r in ml_table.index:
        ml_table.loc[r, c] = np.NaN
        win_prob = win_prob_table.loc[r, c]
        if r == c:
            pass
        elif win_prob > 0.5:
            ml_table.loc[r, c] = -1 * win_prob / (1 - win_prob) * 100
        elif win_prob < 0.5:
            ml_table.loc[r, c] = (1 - win_prob) / win_prob * 100

ml_table = ml_table.applymap(lambda x: '{:+.0f}'.format((x * 2).round(-1) / 2) if not np.isnan(x) else np.NaN)

display(odds_table)
display(ml_table)

,Paul Bartenfeld,Jack Massingill,Evan Sooklal,Roman Ramirez,Lilith Karyadi,Will Simpson,Aaron Carter
Paul Bartenfeld,NaN,0.538523,0.391582,0.306288,0.232292,0.231549,0.075904
Jack Massingill,1.85693,NaN,0.727141,0.568755,0.43135,0.429971,0.140949
Evan Sooklal,2.553742,1.37525,NaN,0.78218,0.593213,0.591317,0.193839
Roman Ramirez,3.264901,1.758226,1.278477,NaN,0.75841,0.755986,0.247819
Lilith Karyadi,4.30493,2.318306,1.685734,1.318548,NaN,0.996804,0.326762
Will Simpson,4.318734,2.325739,1.691139,1.322776,1.003206,NaN,0.327809
Aaron Carter,13.174523,7.094789,5.158909,4.035198,3.060334,3.050552,NaN


,Paul Bartenfeld,Jack Massingill,Evan Sooklal,Roman Ramirez,Lilith Karyadi,Will Simpson,Aaron Carter
Paul Bartenfeld,NaN,+185,+255,+325,+430,+430,+1315
Jack Massingill,-185,NaN,+140,+175,+230,+235,+710
Evan Sooklal,-255,-140,NaN,+130,+170,+170,+515
Roman Ramirez,-325,-175,-130,NaN,+130,+130,+405
Lilith Karyadi,-430,-230,-170,-130,NaN,+100,+305
Will Simpson,-430,-235,-170,-130,-100,NaN,+305
Aaron Carter,-1315,-710,-515,-405,-305,-305,NaN


In [13]:
# best 2 out of 3 calculator
best_23_table = win_pct_table.copy(deep=True)
best_23_table = best_23_table.applymap(lambda x: 0)

for c in best_23_table.columns:
    for r in best_23_table.columns:
        if r == c:
            best_23_table.loc[r, c] = np.NaN
        else:
            win_prob = win_prob_table.loc[r, c]
            best_23_table.loc[r, c] = np.power(win_prob, 2) * (1 + (2 * (1 - win_prob)))

display(best_23_table)

,Paul Bartenfeld,Jack Massingill,Evan Sooklal,Roman Ramirez,Lilith Karyadi,Will Simpson,Aaron Carter
Paul Bartenfeld,NaN,0.281786,0.192984,0.139150,0.093205,0.092756,0.014229
Jack Massingill,0.718214,NaN,0.382498,0.299021,0.217714,0.216864,0.042013
Evan Sooklal,0.807016,0.617502,NaN,0.408791,0.312668,0.311619,0.070528
Roman Ramirez,0.860850,0.700979,0.591209,NaN,0.397605,0.396450,0.102661
Lilith Karyadi,0.906795,0.782286,0.687332,0.602395,NaN,0.498800,0.152092
Will Simpson,0.907244,0.783136,0.688381,0.603550,0.501200,NaN,0.152755
Aaron Carter,0.985771,0.957987,0.929472,0.897339,0.847908,0.847245,NaN


In [14]:
# making playoffs odds
# assumptions
## assume ELO doesn't change after each game
## assume if you win at least 6/8 of your games, you're in for the championship
## independent on the order of matches
def prob_of_x_wins(player, num_wins=6):
    sp = set_players.copy()
    sp.remove(player)
    schedule = list(sp) + list(sp)

    total_prob = 0
    for i in range(np.power(2, 8)):
        single_prob = 1
        record = '{0:08b}'.format(i)
        if record.count('1') != (num_wins):
            continue
        for (j, x) in enumerate(record):
            if x == '1':
                single_prob *= win_prob_table.loc[player, schedule[j]]
            elif x == '0':
                single_prob *= win_prob_table.loc[schedule[j], player]
        total_prob += single_prob
    return total_prob

prob_of_x_wins_table = pd.DataFrame(np.zeros([len(set_players), 9]))
prob_of_x_wins_table.index = list(set_players)
prob_of_x_wins_table.columns
prob_of_x_wins_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), ascending=False, inplace=True)

for c in prob_of_x_wins_table.columns:
    for r in prob_of_x_wins_table.index:
        prob_of_x_wins_table.loc[r, c] = prob_of_x_wins(r, int(c))

odds_of_x_wins_table = (1 - prob_of_x_wins_table) / prob_of_x_wins_table
ml_of_x_wins_table = odds_of_x_wins_table.applymap(lambda x: '{:+,.0f}'.format(float('%.2g' % (x * 100)) if not np.isnan(x) else np.NaN))

# what is the probability of getting at least x wins?
prob_of_cum_x_wins_table = prob_of_x_wins_table.copy(deep=True)
prob_of_cum_x_wins_table.applymap(lambda x: 0)

for (i, c) in enumerate(prob_of_cum_x_wins_table.columns):
    prob_of_cum_x_wins_table.loc[:, c] = prob_of_x_wins_table.iloc[:, i:8+1].sum(axis=1)
prob_of_cum_x_wins_table.drop(columns=0, axis=1, inplace=True)

odds_of_cum_x_wins_table = (1 - prob_of_cum_x_wins_table) / prob_of_cum_x_wins_table
ml_of_cum_x_wins_table = odds_of_cum_x_wins_table.copy(deep=True)

for c in ml_of_cum_x_wins_table.columns:
    for r in ml_of_cum_x_wins_table.index:
        ml_of_cum_x_wins_table.loc[r, c] = np.NaN
        odds = odds_of_cum_x_wins_table.loc[r, c]
        if odds < 1:
            ml_of_cum_x_wins_table.loc[r, c] = -1 * (1 / odds_of_cum_x_wins_table.loc[r, c]) * 100
        elif odds > 1:
            ml_of_cum_x_wins_table.loc[r, c] = odds_of_cum_x_wins_table.loc[r, c] * 100

ml_of_cum_x_wins_table = ml_of_cum_x_wins_table.applymap(lambda x: '{:+,.0f}'.format(float('%.2g' % (x)) if not np.isnan(x) else np.NaN))

display(prob_of_x_wins_table)
display(odds_of_x_wins_table)
display(ml_of_x_wins_table)
display(prob_of_cum_x_wins_table)
display(odds_of_cum_x_wins_table)
display(ml_of_cum_x_wins_table)

,0,1,2,3,4,5,6,7,8
Aaron Carter,1.489277e-07,0.000008,0.000194,0.002480,0.018939,0.088775,0.249862,0.386872,0.252868
Will Simpson,1.922216e-04,0.003390,0.024711,0.096872,0.222368,0.303902,0.238462,0.095837,0.014266
Lilith Karyadi,1.956142e-04,0.003438,0.024976,0.097570,0.223170,0.303883,0.237553,0.095108,0.014106
Roman Ramirez,8.231612e-04,0.010773,0.058001,0.166954,0.279580,0.276913,0.156653,0.045376,0.004927
Evan Sooklal,2.601845e-03,0.026067,0.106928,0.233334,0.294963,0.220017,0.093813,0.020570,0.001707
Jack Massingill,1.352800e-02,0.087166,0.224508,0.301702,0.233380,0.107065,0.028459,0.003974,0.000218
Paul Bartenfeld,1.155025e-01,0.294180,0.317200,0.188685,0.067530,0.014827,0.001937,0.000136,0.000004


,0,1,2,3,4,5,6,7,8
Aaron Carter,6.714665e+06,120239.515683,5146.047566,402.251582,51.799934,10.264426,3.002203,1.584833,2.954627
Will Simpson,5.201330e+03,294.019073,39.467894,9.322886,3.497057,2.290537,3.193542,9.434337,69.094739
Lilith Karyadi,5.111103e+03,289.884839,39.038070,9.249036,3.480879,2.290739,3.209593,9.514352,69.892453
Roman Ramirez,1.213829e+03,91.827531,16.240949,4.989668,2.576799,2.611249,5.383520,21.038265,201.945738
Evan Sooklal,3.833427e+02,37.363169,8.352057,3.285698,2.390252,3.545113,9.659547,47.615237,584.896554
Jack Massingill,7.292078e+01,10.472373,3.454186,2.314524,3.284854,8.340102,34.138404,250.646456,4595.992336
Paul Bartenfeld,7.657819e+00,2.399284,2.152588,4.299841,13.808242,66.443889,515.389558,7364.219202,262463.151887


,0,1,2,3,4,5,6,7,8
Aaron Carter,"+670,000,000","+12,000,000","+510,000","+40,000","+5,200","+1,000",+300,+160,+300
Will Simpson,"+520,000","+29,000","+3,900",+930,+350,+230,+320,+940,"+6,900"
Lilith Karyadi,"+510,000","+29,000","+3,900",+920,+350,+230,+320,+950,"+7,000"
Roman Ramirez,"+120,000","+9,200","+1,600",+500,+260,+260,+540,"+2,100","+20,000"
Evan Sooklal,"+38,000","+3,700",+840,+330,+240,+350,+970,"+4,800","+58,000"
Jack Massingill,"+7,300","+1,000",+350,+230,+330,+830,"+3,400","+25,000","+460,000"
Paul Bartenfeld,+770,+240,+220,+430,"+1,400","+6,600","+52,000","+740,000","+26,000,000"


,1,2,3,4,5,6,7,8
Aaron Carter,1.000000,0.999992,0.999797,0.997317,0.978378,0.889603,0.639741,0.252868
Will Simpson,0.999808,0.996418,0.971707,0.874835,0.652467,0.348566,0.110104,0.014266
Lilith Karyadi,0.999804,0.996367,0.971390,0.873820,0.650650,0.346767,0.109214,0.014106
Roman Ramirez,0.999177,0.988404,0.930403,0.763449,0.483869,0.206956,0.050303,0.004927
Evan Sooklal,0.997398,0.971331,0.864403,0.631069,0.336106,0.116089,0.022276,0.001707
Jack Massingill,0.986472,0.899306,0.674798,0.373096,0.139715,0.032650,0.004191,0.000218
Paul Bartenfeld,0.884497,0.590318,0.273118,0.084433,0.016903,0.002076,0.000140,0.000004


,1,2,3,4,5,6,7,8
Aaron Carter,1.489278e-07,0.000008,0.000203,0.002690,0.022100,0.124097,0.563134,2.954627
Will Simpson,1.922585e-04,0.003595,0.029117,0.143073,0.532643,1.868899,8.082336,69.094739
Lilith Karyadi,1.956525e-04,0.003647,0.029452,0.144400,0.536925,1.883784,8.156338,69.892453
Roman Ramirez,8.238393e-04,0.011732,0.074803,0.309846,1.066675,3.831935,18.879511,201.945738
Evan Sooklal,2.608632e-03,0.029515,0.156868,0.584613,1.975255,7.614074,43.890418,584.896554
Jack Massingill,1.371351e-02,0.111968,0.481925,1.680278,6.157404,29.627634,237.585896,4595.992336
Paul Bartenfeld,1.305855e-01,0.694003,2.661419,10.843681,58.160235,480.671013,7163.179361,262463.151887


,1,2,3,4,5,6,7,8
Aaron Carter,"-670,000,000","-12,000,000","-490,000","-37,000","-4,500",-810,-180,+300
Will Simpson,"-520,000","-28,000","-3,400",-700,-190,+190,+810,"+6,900"
Lilith Karyadi,"-510,000","-27,000","-3,400",-690,-190,+190,+820,"+7,000"
Roman Ramirez,"-120,000","-8,500","-1,300",-320,+110,+380,"+1,900","+20,000"
Evan Sooklal,"-38,000","-3,400",-640,-170,+200,+760,"+4,400","+58,000"
Jack Massingill,"-7,300",-890,-210,+170,+620,"+3,000","+24,000","+460,000"
Paul Bartenfeld,-770,-140,+270,"+1,100","+5,800","+48,000","+720,000","+26,000,000"


In [15]:
fig6 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        "<b>Current Matchup M/L</b><br>Odds for a Single Game", 
        "<b>M/L: Group-Stage Wins</b><br>Odds of Winning at Least <b>X</b> Games"
    ],
    x_title=f"{link_fig}, {link_fig1}, {link_fig2}, {link_fig3}, {link_fig4}, {link_fig5}"
)


go_ml = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in ml_table.columns],
    y=[x.replace(" ", "<br>") for x in ml_table.index],
    z=win_prob_table,
    zmax=max(win_prob_table.max(axis=None), win_prob_table.max(axis=None)),
    zmid=0.5,
    zmin=min(win_prob_table.min(axis=None), win_prob_table.min(axis=None)),
    text=ml_table.applymap(lambda x: "" if math.isnan(float(x)) else x),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

ml_of_cum_x_wins_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), inplace=True)
prob_of_cum_x_wins_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), inplace=True)
scale_go_ml_cum_x_wins = prob_of_cum_x_wins_table

go_ml_cum_x_wins = go.Heatmap(
    x=[(str(x)) for x in ml_of_cum_x_wins_table.columns],
    y=[x.replace(" ", "<br>") for x in ml_of_cum_x_wins_table.index],
    z=scale_go_ml_cum_x_wins,
    zmax=max(scale_go_ml_cum_x_wins.max(axis=None), scale_go_ml_cum_x_wins.max(axis=None)),
    zmid=0,
    zmin=min(scale_go_ml_cum_x_wins.min(axis=None), scale_go_ml_cum_x_wins.min(axis=None)),
    text=ml_of_cum_x_wins_table.applymap(lambda x: "" if math.isnan(float(x.replace(",", "")))  else x.replace(",", "")),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig6.add_trace(go_ml, row=1, col=1)
fig6.add_trace(go_ml_cum_x_wins, row=1, col=2)

fig6.layout.xaxis1.title="<b>Other</b>"
fig6.layout.yaxis1.title="<b>Self</b>"
fig6.layout.xaxis2.title="<b>Number of Total Wins in Group Stage</b>"
fig6.layout.yaxis2.title="<b>Self</b>"

fig6.update_traces(hoverinfo='skip', showscale=False)

fig6.update_layout(
    title=f"<b>Money-Lines</b>, {updated_time}"
)

fig6.show()
fig6.write_html('figures/fig6.html')

In [16]:
ml_table

,Paul Bartenfeld,Jack Massingill,Evan Sooklal,Roman Ramirez,Lilith Karyadi,Will Simpson,Aaron Carter
Paul Bartenfeld,NaN,+185,+255,+325,+430,+430,+1315
Jack Massingill,-185,NaN,+140,+175,+230,+235,+710
Evan Sooklal,-255,-140,NaN,+130,+170,+170,+515
Roman Ramirez,-325,-175,-130,NaN,+130,+130,+405
Lilith Karyadi,-430,-230,-170,-130,NaN,+100,+305
Will Simpson,-430,-235,-170,-130,-100,NaN,+305
Aaron Carter,-1315,-710,-515,-405,-305,-305,NaN


In [17]:
# # system optimization

# def optimization1():
#     index = [_ for _ in np.arange(1, 51, 1.0)]
#     columns = [_ for _ in np.arange(2, 11, 1.0)]

#     opt_table = pd.DataFrame(np.zeros([len(index), len(columns)]))
#     opt_table.index = index
#     opt_table.columns = columns

#     for base in opt_table.columns:
#         for K in opt_table.index:
#             (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)
#             num_upsets = 0
#             for i in elo_time_table.index:
#                 prev_elo_slice = prev_elo_time_table.loc[i,:].dropna().values
#                 elo_slice = elo_time_table.loc[i,:].dropna().values
#                 combo_elo = np.array([prev_elo_slice, elo_slice])
#                 is_upset = ((combo_elo[1, 0] - combo_elo[0, 0]) * (combo_elo[0, 1] - combo_elo[0, 0])) > 0
#                 if is_upset:
#                     num_upsets += 1
#             opt_table.loc[K,base] = num_upsets

#     fig = px.imshow(opt_table.T,labels=dict(y="Base", x="K-value", color="Number of Upsets"))#, text_auto=True, aspect="auto")
#     fig.update_layout(
#         title="<b>Total Number of Upsets against K-Value and Base</b>"
#     )
#     fig.show()
#     return opt_table
# opt1 = optimization1()
# fig.write_html("figures/opt1.html")

In [18]:
# # system optimization

# def optimization2():
#     index = [_ for _ in np.arange(1, 51, 1.0)]
#     columns = [_ for _ in np.arange(2, 11, 1.0)]

#     opt_table = pd.DataFrame(np.zeros([len(index), len(columns)]))
#     opt_table.index = index
#     opt_table.columns = columns

#     for base in opt_table.columns:
#         for K in opt_table.index:
#             (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)
#             total_correctness = 0
#             for i in elo_time_table.index:
#                 d_elo_slice = d_elo_time_table.loc[i,:].dropna().values
#                 exp_elo_slice = exp_elo_time_table.loc[i,:].dropna().values
#                 correctness = (np.sign(d_elo_slice) * exp_elo_slice).sum()
#                 total_correctness += correctness
#             opt_table.loc[K,base] = total_correctness

#     fig = px.imshow(opt_table.T,labels=dict(y="Base", x="K-value", color="Total Correctness"))#, text_auto=False, aspect="auto")
#     fig.update_layout(
#         title="<b>Win Percentage Correctness against K-Value and Base</b>"
#     )
#     fig.show()
#     return opt_table
# opt2 = optimization2()
# fig.write_html("figures/opt2.html")

In [19]:
# def norm_minmax(dataframe):
#     return ((dataframe - dataframe.min(axis=None)) / (dataframe.max(axis=None) - dataframe.min(axis=None)))

# norm_opt1 = norm_minmax(opt1)
# norm_opt2 = norm_minmax(opt2)

# fig = px.imshow(norm_opt2.T - norm_opt1.T,labels=dict(y="Base", x="K-value", color="Normalized Optimization"), text_auto=False, aspect="auto")
# fig.update_layout(
#     title="<b>K Value and Base against Both Optimization Functions</b>"
# )
# fig.show()
# fig.write_html("figures/opt_total.html")